# Rysowanie do PDF

In [1]:
import numpy as np
import pandas as pd
import random
from reportlab.pdfgen import canvas

import generating_cathegories_functions as funs
from puzzle_class import puzzle
from pdf_printing_functions import rysuj_zagadke

In [5]:
kategorie = funs.losuj_kategorie(4, 5, 3, random.randint(100,100000))

puzzle1 = puzzle(4,5)
for i in range(40):
    puzzle1.add_clue1()
    puzzle1.use_clue1(i)
    puzzle1.grid_concile()
    if puzzle1.is_grid_completed() or puzzle1.is_grid_contradictory():
        break

if puzzle1.is_grid_contradictory():
    print("Wyszło sprzeczne!")
        
c = canvas.Canvas("rotate.pdf")
rysuj_zagadke(kategorie, puzzle1.clues, c)
c.showPage()
c.save()

In [6]:
kategorie

[('cathegorical', ['Max', 'Vasco', 'Alvin', 'Pablo', 'Urwis']),
 ('numerical',
  [1500.0, 3000.0, 4500.0, 7500.0, 10500.0],
  {'x=y+1500.0', 'x=y+3000.0', 'x=y+6000.0'},
  '@ CHF'),
 ('numerical',
  [6.0, 12.0, 18.0, 24.0, 30.0],
  {'x=y*2', 'x=y+12.0', 'x=y+18.0', 'x=y+6.0'},
  '@ metrów'),
 ('numerical',
  [3.0, 9.0, 12.0, 18.0, 21.0],
  {'x=y+3.0', 'x=y+6.0', 'x=y+9.0'},
  'Sprawdzian')]

In [3]:
puzzle1 = puzzle(4,5)

for i in range(40):
    puzzle1.add_clue1()
    puzzle1.use_clue1(i)
    puzzle1.grid_concile()
    
    if puzzle1.is_grid_completed() or puzzle1.is_grid_contradictory():
        print(puzzle1.is_grid_completed(), puzzle1.is_grid_contradictory())
        break
puzzle1.print_grid()
print(len(puzzle1.clues))

True False
XXOXX OXXXX XOXXX 
XXXOX XXXXO XXXXO 
XOXXX XOXXX XXOXX 
XXXXO XXOXX OXXXX 
OXXXX XXXOX XXXOX 

XXXXO XXOXX 
XXOXX OXXXX 
XOXXX XOXXX 
OXXXX XXXOX 
XXXOX XXXXO 

XXOXX 
XOXXX 
XXXXO 
OXXXX 
XXXOX 

26


# Inne

In [67]:
def fonts(canvas):
    from reportlab.lib.units import inch
    for font in canvas.getAvailableFonts():
        print(font)
fonts(c)

Courier
Courier-Bold
Courier-BoldOblique
Courier-Oblique
Helvetica
Helvetica-Bold
Helvetica-BoldOblique
Helvetica-Oblique
Symbol
Times-Bold
Times-BoldItalic
Times-Italic
Times-Roman
ZapfDingbats
sans-serif
